In [1]:
from pyspark.sql import SparkSession
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-22"

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
## Read the dataset
training = spark.read.csv('aggregations.csv',header=True,inferSchema=True)

In [4]:
training.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|          Department|
+--------+---+----------+------+--------------------+
|    John| 25|         7| 20000|        Data Science|
| Michael| 28|         9|  8000|    Data Engineering|
|   David| 40|         8| 25000|Software Engineering|
|  Rachel| 29|         6| 18000|  Product Management|
|Jennifer| 31|        10| 12000|Research and Deve...|
|    John| 25|        10| 25000|        Data Science|
| Michael| 28|        12| 12000|    Data Engineering|
|   David| 40|        11| 35000|Software Engineering|
|  Rachel| 29|         9| 25000|  Product Management|
|Jennifer| 31|        13| 18000|Research and Deve...|
+--------+---+----------+------+--------------------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Department: string (nullable = true)



In [7]:
training.columns

['Name', 'Age', 'Experience', 'Salary', 'Department']

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler  =VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Features')

In [9]:
output = featureassembler.transform(training)

In [10]:
output.show()

+--------+---+----------+------+--------------------+--------------------+
|    Name|Age|Experience|Salary|          Department|Independent Features|
+--------+---+----------+------+--------------------+--------------------+
|    John| 25|         7| 20000|        Data Science|          [25.0,7.0]|
| Michael| 28|         9|  8000|    Data Engineering|          [28.0,9.0]|
|   David| 40|         8| 25000|Software Engineering|          [40.0,8.0]|
|  Rachel| 29|         6| 18000|  Product Management|          [29.0,6.0]|
|Jennifer| 31|        10| 12000|Research and Deve...|         [31.0,10.0]|
|    John| 25|        10| 25000|        Data Science|         [25.0,10.0]|
| Michael| 28|        12| 12000|    Data Engineering|         [28.0,12.0]|
|   David| 40|        11| 35000|Software Engineering|         [40.0,11.0]|
|  Rachel| 29|         9| 25000|  Product Management|          [29.0,9.0]|
|Jennifer| 31|        13| 18000|Research and Deve...|         [31.0,13.0]|
+--------+---+----------+

In [11]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Department', 'Independent Features']

In [12]:
finalized_data = output.select("Independent Features", "Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [25.0,7.0]| 20000|
|          [28.0,9.0]|  8000|
|          [40.0,8.0]| 25000|
|          [29.0,6.0]| 18000|
|         [31.0,10.0]| 12000|
|         [25.0,10.0]| 25000|
|         [28.0,12.0]| 12000|
|         [40.0,11.0]| 35000|
|          [29.0,9.0]| 25000|
|         [31.0,13.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
train_data, test_data =finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol='Salary')
regressor = regressor.fit(train_data)

In [15]:
regressor.coefficients

DenseVector([1375.3235, 94.9744])

In [16]:
regressor.intercept

-26347.81823653385

In [17]:
pred_results = regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [25.0,7.0]| 20000| 8700.090279867421|
|         [25.0,10.0]| 25000| 8985.013541979984|
|          [28.0,9.0]|  8000| 13016.00962985243|
|         [28.0,12.0]| 12000|13300.932891964992|
|          [29.0,9.0]| 25000|14391.333132711312|
+--------------------+------+------------------+



In [19]:
pred_results.meanAbsoluteError, pred_results.rootMeanSquaredError

(8848.101113451741, 10232.930593246381)